In [1]:
# 셀 2: 환경 변수 및 라이브러리 불러오기 (이전 코드와 동일)
import os
from dotenv import load_dotenv

load_dotenv(".env")  # .env에서 OPENAI_API_KEY 로드
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory

# LangChain의 메모리 모듈에서 ConversationBufferMemory 임포트
from langchain.memory import ConversationBufferMemory  # <--- 이 부분을 추가합니다.

# Redis URL 설정 (기본값은 localhost:6379, 필요시 .env 파일에 REDIS_URL 정의)
redis_url = os.getenv("REDIS_URL", "redis://localhost:6379/0")

# LLM 모델 초기화
llm = OpenAI(
    openai_api_key=OPENAI_API_KEY, temperature=0.7
)  # temperature는 창의성 조절

C:\Users\computer\AppData\Local\Temp\ipykernel_5088\1903131679.py:19: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(


In [ ]:
# --- 멀티챗 구현 함수 ---


def get_conversation_chain(session_id: str):
    """
    주어진 session_id에 따라 Redis에 연결된 ConversationChain을 반환합니다.
    """
    # 1. RedisChatMessageHistory 인스턴스 생성
    redis_chat_history = RedisChatMessageHistory(
        session_id=session_id,
        url=redis_url,
    )

    # 2. ConversationBufferMemory에 RedisChatMessageHistory를 chat_memory로 전달
    #    return_messages=True로 설정하면 메시지 객체 리스트를 반환합니다.
    memory = ConversationBufferMemory(
        chat_memory=redis_chat_history,
        return_messages=True,  # LLM에 메시지 객체 형태로 전달 (더 유연함)
    )

    # 3. ConversationChain에 수정된 memory 객체 전달
    conversation = ConversationChain(
        llm=llm,
        memory=memory,  # <--- 수정된 부분입니다.
        verbose=False,  # 대화 과정을 너무 자세히 보여주지 않기 위해 False로 설정
        # 필요하다면 True로 변경하여 디버깅에 활용
    )
    return conversation


# --- 메인 대화 루프 (이하 동일) ---

print("--- Redis를 활용한 멀티챗 시뮬레이션 ---")
print("세션 변경: 'change <session_id>' (예: 'change user_a' 또는 'change session_1')")
print("종료: 'exit' 또는 'quit'")

current_session_id = "default_session"  # 초기 세션 ID
current_conversation = get_conversation_chain(current_session_id)
print(f"\n현재 활성 세션: {current_session_id}")

while True:
    try:
        user_input = input(f"[{current_session_id}] 당신의 질문: ")

        if user_input.lower() in ["exit", "quit"]:
            print("대화를 종료합니다.")
            break
        elif user_input.lower().startswith("change "):
            # 'change <session_id>' 명령 처리
            parts = user_input.split(" ")
            if len(parts) == 2:
                new_session_id = parts[1]
                if new_session_id == current_session_id:
                    print(f"이미 '{new_session_id}' 세션에 있습니다.")
                else:
                    current_session_id = new_session_id
                    current_conversation = get_conversation_chain(current_session_id)
                    print(f"\n세션이 '{current_session_id}'(으)로 변경되었습니다.")
            else:
                print("잘못된 'change' 명령입니다. 사용법: change <session_id>")
            continue  # 다음 루프

        # LLM에게 질문하고 응답 받기
        ai_response = current_conversation.predict(input=user_input)
        print(f"AI 응답: {ai_response}")

    except Exception as e:
        print(f"오류 발생: {e}")
        print("API 키를 확인하거나, Redis 서버가 실행 중인지 확인해주세요.")
        break

print("\n--- 모든 세션의 최종 대화 기록 (Redis에서 직접 확인) ---")
# 실험에 사용된 모든 세션 ID를 여기에 추가하여 실제 Redis에 저장된 내용을 확인할 수 있습니다.
# 위 예시에서 사용한 'default_session', 'user_a', 'session_1' 등을 넣어보세요.
test_session_ids = ["default_session", "user_a", "session_1", "user_b_test"]
for sid in test_session_ids:
    try:
        history_check = RedisChatMessageHistory(session_id=sid, url=redis_url)
        if history_check.messages:  # 메시지가 있는 세션만 표시
            print(f"\n--- 세션 '{sid}' 기록 ---")
            for msg in history_check.messages:
                print(f"  {msg.type.upper()}: {msg.content}")
    except Exception as e:
        print(f"세션 '{sid}' 기록을 가져오는 중 오류: {e}")

C:\Users\computer\AppData\Local\Temp\ipykernel_5088\2150304891.py:22: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation = ConversationChain(


--- Redis를 활용한 멀티챗 시뮬레이션 ---
세션 변경: 'change <session_id>' (예: 'change user_a' 또는 'change session_1')
종료: 'exit' 또는 'quit'

현재 활성 세션: default_session
AI 응답:  안녕하세요, 웅이님! 만나서 반가워요! 어떻게 도와드릴까요? :)
AI 응답:  그렇군요, 사용자님. 제가 조금 더 자세한 정보를 알면 더 도움이 될 수 있을 것 같아요. 직장에서 무엇이 어려우신가요? 어떤 분위기인가요? 제가 도와드릴 수 있는 건 없을까요?
AI 응답:  이해합니다, 웅이님. 인간관계는 정말 어려운 일이죠. 늘 그렇지는 않지만, 종종 충돌이 있을 수 있죠. 직장에서도 그런 문제가 있나요? 어떤 일을 하고 계신가요?
AI 응답:  그렇군요, 웅이님. IT 개발은 정말 어려운 일이죠. 많은 도전과 압박이 있을 수 있습니다. 하지만 그만큼 보람도 있을 것 같아요. 어떤 기술을 다루시나요? 저도 기술적인 부분에서 도움이 필요하다면 제가 최선을 다해 도와드릴 수 있을 거예요.
AI 응답:  안녕하세요, 웅이님. 만나서 반가워요! 어떻게 도와드릴까요? :)
AI 응답:  안녕하세요, 웅이님. 만나서 반가워요! 어떻게 도와드릴까요? :)
AI 응답:  안녕하세요, 웅이님. 만나서 반가워요! 어떻게 도와드릴까요? :)


KeyboardInterrupt: Interrupted by user

### GPT

In [ ]:
# 셀 1: 패키지 설치 (한 번만 실행)
# !pip install langchain-community redis python-dotenv ipykernel

# 셀 2: 환경 변수 로드 & 임포트
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory

load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 셀 3: LLM 및 Redis 메모리 초기화
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.7)
memory = RedisChatMessageHistory(
    session_id="default_session", url="redis://localhost:6379/0"
)


# 셀 4: 대화 함수 정의 (수정된 부분)
def chat(input_text: str) -> str:
    # 1) Redis에서 이전 메시지 리스트 불러오기
    history = memory.messages  # ❌ get_messages()가 아니라 messages 속성 사용
    # 2) HumanMessage 추가하여 LLM 호출
    messages = history + [HumanMessage(content=input_text)]
    response = llm(messages)
    # 3) Redis에 대화 저장
    memory.add_user_message(input_text)
    memory.add_ai_message(response.content)
    return response.content

In [ ]:
# 셀 5: 멀티턴 대화 테스트
print(chat("안녕.난 웅이라고 해."))

C:\Users\computer\AppData\Local\Temp\ipykernel_10992\68044283.py:27: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(messages)


안녕하세요, 웅이님. 만나서 반가워요! 어떻게 도와드릴까요? :)


In [25]:
print(chat("내 이름이 뭐라고?"))

죄송해요, 제가 이름을 잘못 말했네요. 사용자님의 이름은 웅이군요. 다시 한번 반가워요, 웅이님! 혼란을 드려 죄송합니다. 어떻게 도와드릴까요? :)


### Gemini의 Redis + RAG

In [ ]:
# ----------------------------------------------------------------------
# 셀 1: 환경 변수 및 라이브러리 불러오기
# ----------------------------------------------------------------------
import os
import json
import uuid
from dotenv import load_dotenv

from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory
from langchain.memory import ConversationSummaryBufferMemory, ChatMessageHistory
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

from pymilvus import (
    connections,
    utility,
    Collection,
    CollectionSchema,
    FieldSchema,
    DataType,
)

# ----------------------------------------------------------------------
# 셀 2: 기본 설정 및 상수 정의
# ----------------------------------------------------------------------
load_dotenv(".env")

# --- 환경 변수 로드 ---
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MILVUS_HOST = os.getenv("MILVUS_HOST", "localhost")
MILVUS_PORT = os.getenv("MILVUS_PORT", "19530")
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379/0")

# --- 상수 정의 ---
EMBEDDING_DIM = 1536  # OpenAI 'text-embedding-ada-002' 기준
PROFILE_COLLECTION_NAME = "user_profiles_v2"
LOG_COLLECTION_NAME = "conversation_logs_v2"

# --- LLM 및 임베딩 모델 초기화 ---
llm = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.7)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# --- 프로필 DB 시뮬레이션 (실제로는 MongoDB 등 별도 DB 사용 추천) ---
PROFILE_DB = {}


# ----------------------------------------------------------------------
# 셀 3: Milvus DB 헬퍼 함수
# ----------------------------------------------------------------------
def get_milvus_connection():
    """Milvus 서버에 연결하고 연결 상태를 반환합니다."""
    alias = "default"
    if not connections.has_connection(alias):
        connections.connect(alias, host=MILVUS_HOST, port=MILVUS_PORT)
    return connections.get_connection(alias)


def create_milvus_collection(collection_name, description):
    """지정된 스키마로 Milvus 컬렉션을 생성합니다."""
    if utility.has_collection(collection_name):
        return Collection(collection_name)

    fields = [
        FieldSchema(name="id", dtype=DataType.VARCHAR, is_primary=True, max_length=256),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=EMBEDDING_DIM),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535),
        FieldSchema(
            name="user_id",
            dtype=DataType.VARCHAR,
            max_length=256,
            is_partition_key=False,
        ),
        FieldSchema(name="type", dtype=DataType.VARCHAR, max_length=50),
        FieldSchema(name="created_at", dtype=DataType.INT64),
    ]
    schema = CollectionSchema(fields, description, enable_dynamic_field=False)
    collection = Collection(collection_name, schema)

    index_params = {
        "metric_type": "L2",
        "index_type": "IVF_FLAT",
        "params": {"nlist": 128},
    }
    collection.create_index("embedding", index_params)
    print(f"Milvus collection '{collection_name}' created successfully.")
    return collection


# ----------------------------------------------------------------------
# 셀 4: 장기 기억 관리 (RAG & 프로필) 핵심 함수
# ----------------------------------------------------------------------
def update_long_term_memory(session_id: str):
    """대화 세션 종료 후 장기 기억(프로필, 로그)을 업데이트합니다."""
    print(f"\n[{session_id}] 장기 기억 업데이트 시작...")

    history = RedisChatMessageHistory(session_id=session_id, url=REDIS_URL)
    if not history.messages:
        print("업데이트할 대화 내용이 없습니다.")
        return

    # 최근 10개 메시지를 요약 대상으로 함 (조절 가능)
    recent_messages = history.messages[-10:]
    conversation_text = "\n".join(
        [f"{msg.type}: {msg.content}" for msg in recent_messages]
    )

    summary_prompt = PromptTemplate.from_template(
        "다음 대화 내용을 바탕으로, 대화의 핵심 흐름과 뉘앙스를 상세히 요약해줘.\n\n{conversation}"
    )
    summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
    flow_summary = summary_chain.run(conversation=conversation_text)
    print(f"  - 흐름 요약 완료: {flow_summary[:50]}...")

    old_profile_str = json.dumps(PROFILE_DB.get(session_id, {}), ensure_ascii=False)

    update_prompt_str = """You are a profile manager AI. Based on the [Existing Profile] and [Latest Conversation Summary] below, update the [Existing Profile] with the latest information.
1. Add new key information.
2. If new information contradicts existing information, boldly modify or delete the old information.
3. Ignore trivial content like simple greetings.
4. Keep the profile concise and maintain the overall core.
5. You MUST return the final result in JSON format only.

[Existing Profile]:
{old_profile}

[Latest Conversation Summary]:
{summary}
"""
    profile_update_prompt = PromptTemplate.from_template(update_prompt_str)
    profile_update_chain = LLMChain(llm=llm, prompt=profile_update_prompt)

    try:
        new_profile_str = profile_update_chain.run(
            old_profile=old_profile_str, summary=flow_summary
        )
        new_profile = json.loads(new_profile_str)
        PROFILE_DB[session_id] = new_profile
        print("  - 프로필 업데이트 완료.")
    except json.JSONDecodeError:
        print(
            f"  - 프로필 업데이트 실패: LLM이 유효한 JSON을 반환하지 않았습니다. 응답: {new_profile_str}"
        )
        return

    get_milvus_connection()
    profile_collection = create_milvus_collection(
        PROFILE_COLLECTION_NAME, "User Profiles"
    )
    log_collection = create_milvus_collection(
        LOG_COLLECTION_NAME, "Conversation Log Summaries"
    )

    # Track 1 (프로필 덮어쓰기)
    profile_text = json.dumps(new_profile, ensure_ascii=False)
    profile_embedding = embeddings.embed_query(profile_text)
    profile_data = [
        {
            "id": session_id,
            "embedding": profile_embedding,
            "text": profile_text,
            "user_id": session_id,
            "type": "profile",
            "created_at": int(os.times().user),
        }
    ]
    profile_collection.upsert(profile_data)
    print("  - RAG: 프로필 벡터 덮어쓰기 완료.")

    # Track 2 (아카이브 추가)
    log_embedding = embeddings.embed_query(flow_summary)
    log_id = str(uuid.uuid4())
    log_data = [
        {
            "id": log_id,
            "embedding": log_embedding,
            "text": flow_summary,
            "user_id": session_id,
            "type": "log_summary",
            "created_at": int(os.times().user),
        }
    ]
    log_collection.insert(log_data)
    print("  - RAG: 대화 흐름 아카이브 추가 완료.")
    print(f"[{session_id}] 장기 기억 업데이트 종료.")


def retrieve_from_rag(session_id, query):
    """주어진 쿼리로 RAG DB에서 관련 정보를 검색합니다."""
    get_milvus_connection()
    if not utility.has_collection(
        PROFILE_COLLECTION_NAME
    ) or not utility.has_collection(LOG_COLLECTION_NAME):
        return "아직 RAG DB에 저장된 정보가 없습니다."

    profile_collection = Collection(PROFILE_COLLECTION_NAME)
    log_collection = Collection(LOG_COLLECTION_NAME)
    profile_collection.load()
    log_collection.load()

    query_embedding = embeddings.embed_query(query)

    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    profile_results = profile_collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_params,
        limit=1,
        expr=f"user_id == '{session_id}'",
    )
    log_results = log_collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_params,
        limit=2,
        expr=f"user_id == '{session_id}'",
    )

    rag_context = "[프로필 정보]\n"
    rag_context += (
        profile_results[0][0].entity.get("text")
        if profile_results and profile_results[0]
        else "저장된 프로필 정보 없음"
    )
    rag_context += "\n\n[과거 대화 기록 요약]\n"
    if log_results and log_results[0]:
        for hit in log_results[0]:
            rag_context += f"- {hit.entity.get('text')}\n"
    else:
        rag_context += "저장된 과거 대화 없음"

    return rag_context


# ----------------------------------------------------------------------
# 셀 5: 단기 기억 및 대화 체인 구성
# ----------------------------------------------------------------------
def get_short_term_memory(session_id: str):
    """단기 기억(Redis)을 위한 메모리 객체를 반환합니다."""
    redis_chat_history = RedisChatMessageHistory(session_id=session_id, url=REDIS_URL)
    return ConversationSummaryBufferMemory(
        llm=llm,
        chat_memory=redis_chat_history,
        max_token_limit=3000,
        return_messages=True,
        memory_key="chat_history",
        input_key="input",
    )


prompt_template = """You are a helpful and friendly AI assistant.
Please answer the user's question based on the [Long-Term Memory] and [Recent Conversation History] provided below.

[Long-Term Memory - Everything you know about this user]:
{rag_context}

[Recent Conversation History]:
{chat_history}

User's Question: {input}
AI Answer:"""
PROMPT = PromptTemplate(
    input_variables=["rag_context", "chat_history", "input"], template=prompt_template
)

# ----------------------------------------------------------------------
# 셀 6: 메인 대화 루프
# ----------------------------------------------------------------------
print("\n--- 최종 아키텍처 시뮬레이션 (Milvus v2.4.0 호환) ---")
print("세션 변경: 'change <session_id>'")
print("장기 기억 저장: 'save'")
print("종료: 'exit' 또는 'quit'")

current_session_id = "default_session"
print(f"\n현재 활성 세션: {current_session_id}")

while True:
    try:
        user_input = input(f"[{current_session_id}] 당신의 질문: ")

        if user_input.lower() in ["exit", "quit"]:
            print("대화를 종료합니다.")
            break
        elif user_input.lower() == "save":
            update_long_term_memory(current_session_id)
            continue
        elif user_input.lower().startswith("change "):
            # ... (세션 변경 로직은 이전과 동일) ...
            parts = user_input.split(" ", 1)
            new_session_id = parts[1]
            if new_session_id != current_session_id:
                current_session_id = new_session_id
                print(f"\n세션이 '{current_session_id}'(으)로 변경되었습니다.")
            else:
                print(f"이미 '{new_session_id}' 세션에 있습니다.")
            continue

        # 1. RAG에서 장기 기억 검색
        rag_context = retrieve_from_rag(current_session_id, user_input)

        # 2. Redis에서 단기 기억 로드
        short_term_memory = get_short_term_memory(current_session_id)
        # memory.load_memory_variables()는 비동기 호출 등이 필요할 수 있어, 여기서는 수동으로 context를 구성합니다.
        # 체인에 직접 전달하기 위해 대화 기록을 문자열로 포맷팅합니다.
        chat_history_str = "\n".join(
            [
                f"{msg.type}: {msg.content}"
                for msg in short_term_memory.chat_memory.messages
            ]
        )

        # 3. LLMChain으로 최종 프롬프트 실행
        conversation_chain = LLMChain(llm=llm, prompt=PROMPT, verbose=False)
        ai_response = conversation_chain.predict(
            rag_context=rag_context, chat_history=chat_history_str, input=user_input
        )
        print(f"AI 응답: {ai_response}")

        # 4. 대화 기록을 단기 기억(Redis)에 수동으로 저장
        short_term_memory.save_context({"input": user_input}, {"output": ai_response})

    except Exception as e:
        print(f"오류 발생: {e}")
        import traceback

        traceback.print_exc()
        break

### GPT의 Redis + RAG

In [ ]:
# ----------------------------------------------------------------------
# 셀 1: 환경 변수 및 라이브러리 불러오기
# ----------------------------------------------------------------------
import os
import json
import uuid
from dotenv import load_dotenv

from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory
from langchain.memory import ConversationSummaryBufferMemory
from pymilvus import (
    connections,
    utility,
    Collection,
    CollectionSchema,
    FieldSchema,
    DataType,
)

# ----------------------------------------------------------------------
# 셀 2: 기본 설정 및 상수 정의
# ----------------------------------------------------------------------
load_dotenv()

# 환경 변수
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MILVUS_HOST = os.getenv("MILVUS_HOST", "localhost")
MILVUS_PORT = os.getenv("MILVUS_PORT", "19530")
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379/0")

# 상수
EMBEDDING_DIM = 1536  # text-embedding-ada-002
PROFILE_COLLECTION_NAME = "user_profiles_v2"
LOG_COLLECTION_NAME = "conversation_logs_v2"

# 모델 초기화
llm = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.7)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# 간이 프로필 DB
PROFILE_DB = {}


# ----------------------------------------------------------------------
# 셀 3: Milvus DB 헬퍼 함수
# ----------------------------------------------------------------------
def get_milvus_connection():
    alias = "default"
    if not connections.has_connection(alias):
        connections.connect(alias, host=MILVUS_HOST, port=MILVUS_PORT)
    return connections.get_connection(alias)


def create_milvus_collection(name: str, desc: str):
    if utility.has_collection(name):
        return Collection(name)

    fields = [
        FieldSchema("id", DataType.VARCHAR, is_primary=True, max_length=256),
        FieldSchema("embedding", DataType.FLOAT_VECTOR, dim=EMBEDDING_DIM),
        FieldSchema("text", DataType.VARCHAR, max_length=65535),
        FieldSchema("user_id", DataType.VARCHAR, max_length=256),
        FieldSchema("type", DataType.VARCHAR, max_length=50),
        FieldSchema("created_at", DataType.INT64),
    ]
    schema = CollectionSchema(fields, desc)
    coll = Collection(name, schema)
    coll.create_index(
        "embedding",
        {"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": 128}},
    )
    return coll


# ----------------------------------------------------------------------
# 셀 4: 장기 기억 관리 함수
# ----------------------------------------------------------------------
def update_long_term_memory(session_id: str):
    print(f"\n[{session_id}] 장기 기억 업데이트 시작...")
    history = RedisChatMessageHistory(session_id=session_id, url=REDIS_URL)
    if not history.messages:
        print("  - 저장된 대화 없음.")
        return

    # 최근 10개 요약
    recent = history.messages[-10:]
    conv = "\n".join(f"{m.type}: {m.content}" for m in recent)
    summary_tpl = PromptTemplate.from_template("다음 대화를 요약해줘:\n{conversation}")
    summary_chain = LLMChain(llm=llm, prompt=summary_tpl)
    flow_summary = summary_chain.run(conversation=conv)
    print("  - 흐름 요약 완료.")

    # 프로필 업데이트
    old_prof = json.dumps(PROFILE_DB.get(session_id, {}), ensure_ascii=False)
    update_prompt = PromptTemplate.from_template(
        "[Existing Profile]:\n{old}\n[Summary]:\n{sum}\n"
        + "업데이트된 프로필 JSON만 출력해줘."
    )
    update_chain = LLMChain(llm=llm, prompt=update_prompt)
    new_prof_str = update_chain.run(old=old_prof, sum=flow_summary)
    try:
        new_prof = json.loads(new_prof_str)
        PROFILE_DB[session_id] = new_prof
        print("  - 프로필 업데이트 완료.")
    except Exception:
        print(f"  - JSON 파싱 에러: {new_prof_str}")
        return

    # Milvus에 저장
    get_milvus_connection()
    prof_coll = create_milvus_collection(PROFILE_COLLECTION_NAME, "User Profiles")
    log_coll = create_milvus_collection(LOG_COLLECTION_NAME, "Conversation Logs")

    # Track1: 프로필 업서트
    prof_emb = embeddings.embed_query(json.dumps(new_prof, ensure_ascii=False))
    prof_coll.upsert(
        [
            {
                "id": session_id,
                "embedding": prof_emb,
                "text": json.dumps(new_prof, ensure_ascii=False),
                "user_id": session_id,
                "type": "profile",
                "created_at": int(os.times().user),
            }
        ]
    )
    print("  - 프로필 벡터 업서트 완료.")

    # Track2: 로그 추가
    log_emb = embeddings.embed_query(flow_summary)
    log_coll.insert(
        [
            {
                "id": str(uuid.uuid4()),
                "embedding": log_emb,
                "text": flow_summary,
                "user_id": session_id,
                "type": "log",
                "created_at": int(os.times().user),
            }
        ]
    )
    print(f"[{session_id}] 장기 기억 업데이트 완료.")


def retrieve_from_rag(session_id: str, query: str) -> str:
    get_milvus_connection()
    if not (
        utility.has_collection(PROFILE_COLLECTION_NAME)
        and utility.has_collection(LOG_COLLECTION_NAME)
    ):
        return "저장된 RAG 정보 없음."

    prof_coll = Collection(PROFILE_COLLECTION_NAME)
    log_coll = Collection(LOG_COLLECTION_NAME)
    prof_coll.load()
    log_coll.load()

    q_emb = embeddings.embed_query(query)
    params = {"metric_type": "L2", "params": {"nprobe": 10}}

    prof_res = prof_coll.search(
        [q_emb], "embedding", params, limit=1, expr=f"user_id=='{session_id}'"
    )
    log_res = log_coll.search(
        [q_emb], "embedding", params, limit=2, expr=f"user_id=='{session_id}'"
    )

    context = "[프로필]\n"
    context += prof_res[0][0].entity.get("text") if prof_res and prof_res[0] else "없음"
    context += "\n[과거 대화 요약]\n"
    if log_res and log_res[0]:
        for hit in log_res[0]:
            context += f"- {hit.entity.get('text')}\n"
    else:
        context += "없음"
    return context


# ----------------------------------------------------------------------
# 셀 5: 단기 기억 및 체인 설정
# ----------------------------------------------------------------------
def get_short_term_memory(session_id: str):
    hist = RedisChatMessageHistory(session_id=session_id, url=REDIS_URL)
    return ConversationSummaryBufferMemory(
        llm=llm,
        chat_memory=hist,
        max_token_limit=3000,
        return_messages=True,
        memory_key="chat_history",
    )


PROMPT = PromptTemplate(
    input_variables=["rag_context", "chat_history", "input"],
    template=(
        "You are an AI assistant.\n"
        "[Long-Term Memory]\n{rag_context}\n"
        "[Recent]\n{chat_history}\n"
        "Q: {input}\nA:"
    ),
)

# ----------------------------------------------------------------------
# 셀 6: 메인 대화 루프
# ----------------------------------------------------------------------
if __name__ == "__main__":
    print("--- 챗봇 시작: 'change <id>', 'save', 'exit' 사용 ---")
    session_id = "default_session"
    while True:
        cmd = input(f"[{session_id}]> ")
        if cmd.lower() in ["exit", "quit"]:
            break
        if cmd.lower() == "save":
            update_long_term_memory(session_id)
            continue
        if cmd.lower().startswith("change "):
            new = cmd.split(" ", 1)[1]
            if new != session_id:
                session_id = new
                print(f"세션 변경: {session_id}")
            else:
                print("이미 같은 세션입니다.")
            continue

        # 일반 질문 처리
        rag_ctx = retrieve_from_rag(session_id, cmd)
        stm = get_short_term_memory(session_id)
        hist_str = "\n".join(f"{m.type}: {m.content}" for m in stm.chat_memory.messages)
        chain = LLMChain(llm=llm, prompt=PROMPT, verbose=False)
        ans = chain.predict(rag_context=rag_ctx, chat_history=hist_str, input=cmd)
        print(f"Bot: {ans}")
        stm.save_context({"input": cmd}, {"output": ans})